# Trump Tweet Bot (Keras)
---


In [1]:
%load_ext lab_black
import sys, random, os, gc, re, json
from string import punctuation

import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dropout, Input, Dense, LSTM

from eda_utils import tweet_cleaner, rmv_uncommon
from bot_utils import DataGenerator, TextCorpus, load_corpus

### Load and Clean Data
----


In [2]:
DATA_FILE = "./data/archive.json"

corpus = load_corpus(DATA_FILE)

### Pipeline Set-up
----

In [3]:
full_corp = TextCorpus(corpus)

train_corp = full_corp[: int(len(full_corp) * 0.9)]
test_corp = full_corp[int(len(full_corp) * 0.9) :]

SAMPLE_LEN = 32
STEP_SIZE = 3
BATCH_SIZE = 32

train_gen = DataGenerator(train_corp, SAMPLE_LEN, STEP_SIZE)
test_gen = DataGenerator(test_corp, SAMPLE_LEN, STEP_SIZE, shuffle=False)

nchars = full_corp.get_num_chars()

model = Sequential(name="test_bot1")
model.add(Input(shape=(SAMPLE_LEN, nchars), dtype=np.float32))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(nchars, activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam")
print(model.summary())

Model: "test_bot1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 32, 256)           345088    
_________________________________________________________________
dropout (Dropout)            (None, 32, 256)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 80)                20560     
Total params: 890,960
Trainable params: 890,960
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
old_encode(corpus, full_corp.char_to_indicies)
%timeit old_encode(corpus, full_corp.char_to_indicies)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
non-precise type pyobject
During: typing of argument at <ipython-input-11-4532fece08dc> (6)

File "<ipython-input-11-4532fece08dc>", line 6:
def old_encode(text, mapping):
    data = np.zeros((len(text),), dtype=np.int32)
    ^

This error may have been caused by the following argument(s):
- argument 1: cannot determine Numba type of <class 'dict'>


In [7]:
%timeit full_corp.encode_numerical(corpus)
%timeit old_encode(corpus)

352 ms ± 5.02 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
474 ms ± 7.31 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Train an Predict
---

In [ ]:
tweet_ends = np.where(np.asarray(list(corpus)) == "@")[0]

for epoch in range(1, 31):
    print('-' * 40)
    print('Epoch', epoch)
    model.fit(
        train_gen,
        steps_per_epoch=train_gen.epoch_size,
        epochs=1,
        validation_data=test_gen,
        validation_steps=test_gen.epoch_size)
    
    seed_index = 1 + np.random.choice(tweet_ends, 1)[0]
    for diversity in [0.2, 0.7, 1.2]:
        genertate_tweet(model, seed_index, diversity, full_corp)
    print('-' * 40)    

----------------------------------------
Epoch 1
